# COS 710 - Artificial Intelligence I
## Assignment 2: Classifiers

In [ ]:
# install Anaconda3
!wget -qO ac.sh https://repo.anaconda.com/archive/Anaconda3-2020.07-Linux-x86_64.sh 
!bash ./ac.sh -b

# a fake google.colab library
!ln -s /usr/local/lib/python3.6/dist-packages/google \
       /root/anaconda3/lib/python3.8/site-packages/google

# start jupyterlab, which now has Python3 = 3.8
!nohup /root/anaconda3/bin/jupyter-lab --ip=0.0.0.0&

# access through ngrok, click the link
!pip install pyngrok -q
from pyngrok import ngrok
print(ngrok.connect(8888))

ERROR: File or directory already exists: '/root/anaconda3'
If you want to update an existing installation, use the -u option.
nohup: appending output to 'nohup.out'


t=2020-09-28T12:54:48+0000 lvl=warn msg="failed to start tunnel" pg=/api/tunnels id=7cff9d603a709589 err="Your account may not run more than 2 tunnels over a single ngrok client session.\nThe tunnels already running on this session are:\n[https://8005cc85d87f.ngrok.io http://8005cc85d87f.ngrok.io]\n\r\n\r\nERR_NGROK_324\r\n"



PyngrokNgrokHTTPError: ignored

In [1]:
!pip install PyDrive
!pip install gplearn
!pip install git+https://github.com/pySTEPS/pysteps
!pip install Pyvolution
!pip install decision-tree-id3
!pip install pyshgp
!pip install deap
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import math
import gc
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

%pylab inline
from gplearn.genetic import SymbolicRegressor
from gplearn.genetic import SymbolicClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils.random import check_random_state
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import graphviz
from sklearn.datasets import load_breast_cancer
from id3 import Id3Estimator
from id3 import export_graphviz
from sklearn.model_selection import train_test_split

#from pyshgp.push.instruction_set import InstructionSet

#from pyshgp.gp.estimators import PushEstimator
#from pyshgp.gp.genome import GeneSpawner

     |████████████████████████████████| 51kB 2.2MB/s 
  Cloning https://github.com/pySTEPS/pysteps to /tmp/pip-req-build-uj6gyt4a
  Running command git clone -q https://github.com/pySTEPS/pysteps /tmp/pip-req-build-uj6gyt4a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
Processing /root/.cache/pip/wheels/64/f4/de/9667d84f759289edf5442220997c6d4334637a6bb2a7b90f73/jsmin-2.2.2-cp36-none-any.whl
  Created wheel for pysteps: filename=pysteps-1.3.2-cp36-cp36m-linux_x86_64.whl size=965718 sha256=99e5bb06ee7a23a9091e10621d115fb0883afddf0d33311d75f15ad3a17cee27
  Stored in directory: /tmp/pip-ephem-wheel-cache-n5l5qx1r/wheels/22/e9/c0/c19c043836ff5fbe1697cb0a0a4d328b8a6d85cf552cb50f5d
Successfully built pysteps
  Created wheel for Pyvolution: filename=Pyvolution-1.1-cp36-none-any.whl size=15360 sha256=04e6760b724be2e2e49854141c08b2397a1a394115b99fdb047404678361075e
  Stored

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [57]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
drive.CreateFile({'id':"1NapWxZHdXFJ3wdNAHjiUXO803fjw-lF3"}).GetContentFile("post-operative.csv")
input_df = pd.read_csv("post-operative.csv")


In [109]:
# encode the data
from sklearn import preprocessing
l_encoder = preprocessing.LabelEncoder()
oh_encoder = preprocessing.OneHotEncoder(sparse=False)
or_encoder = preprocessing.OrdinalEncoder( categories= [array(['low', 'mid', 'high'], dtype=object), array(['low', 'mid', 'high',], dtype=object), \
                                                        array(['poor', 'fair', 'good','excellent', ], dtype=object), \
                                                        array(['low', 'mid','high',], dtype=object), array(['unstable','stable' ], dtype=object), \
                                                        array([ 'unstable', 'mod-stable','stable'], dtype=object), array(['unstable', 'mod-stable', 'stable' ], dtype=object), \
                                                        array(['?', '1', '2','3', '4', '5', '6', '7', '8', '9','10','11','12', '13', '14', '15','16', '17', '18', '19','20'], dtype=object)])

train_df, test_df = train_test_split(input_df, test_size=0.2)

y_train = train_df["ADM-DECS"]
X_train = train_df.drop(columns=["ADM-DECS"])

feat_names = X_train.columns

label_train = l_encoder.fit_transform(y_train)
# print(y_train)

ordinal_train = or_encoder.fit_transform(X_train)
onehot_train = oh_encoder.fit_transform(X_train)
print(ordinal_train.shape)
print(label_train.size)


(70, 8)
70


In [100]:
print(label_train)
print(y_train.values)

[0 1 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1 1 0 0 0 1 1 0 0 1 0 0 0 0 1 0
 0 0 0 0 1 0 0 1 0 0 1 0 1 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 1 0]
['A' 'S' 'A' 'A' 'S' 'A' 'A' 'A' 'A' 'A' 'A' 'S' 'A' 'S' 'A' 'A' 'A' 'S'
 'A' 'A' 'A' 'S' 'S' 'A' 'A' 'A' 'S' 'S' 'A' 'A' 'S' 'A' 'A' 'A' 'A' 'S'
 'A' 'A' 'A' 'A' 'A' 'S' 'A' 'A' 'S' 'A' 'A' 'S' 'A' 'S' 'S' 'A' 'S' 'A'
 'A' 'A' 'A' 'A' 'S' 'A' 'A' 'A' 'S' 'A' 'S' 'A' 'A' 'A' 'S' 'A']


In [110]:
# Use a symbolic classifier>

est = SymbolicClassifier(population_size=400, parsimony_coefficient=.01,
                         feature_names=feat_names,
                         random_state=99,
                         generations=30, tournament_size=3, stopping_criteria=0,
                           p_crossover=0.8, p_subtree_mutation=0.2,
                           p_hoist_mutation=0.0, p_point_mutation=0,
                           const_range=(-1,1), init_method="half and half",
                           max_samples=0.9, verbose=1, init_depth=(2,6),
                           )
est.fit(ordinal_train, label_train)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    27.39          4.66424        7         0.561563         0.821569     13.11s
   1    14.78          1.89594       13         0.549446         0.616887     10.95s
   2     8.52          1.43394        1         0.551413          1.02675     10.05s
   3     6.88          1.54645        9         0.551738          1.25169      9.51s
   4     6.52           1.4217        5         0.537209          1.29298      9.31s
   5     5.86          1.34195        5         0.526931          0.97355      8.92s
   6     4.94          1.14211        3         0.543988         0.940098      8.50s
   7     4.27          1.14662        3          0.50777          1.40404      8.08s
   8     3.73          1.07949        3         0.506573          1.27683  

SymbolicClassifier(const_range=(-1, 1),
                   feature_names=Index(['L-CORE', 'L-SURF', 'L-O2', 'L-BP', 'SURF-STBL', 'CORE-STBL', 'BP-STBL',
       'COMFORT'],
      dtype='object'),
                   function_set=('add', 'sub', 'mul', 'div'), generations=30,
                   init_depth=(2, 6), init_method='half and half',
                   low_memory=False, max_samples=0.9, metric='log loss',
                   n_jobs=1, p_crossover=0.8, p_hoist_mutation=0.0,
                   p_point_mutation=0, p_point_replace=0.05,
                   p_subtree_mutation=0.2, parsimony_coefficient=0.01,
                   population_size=400, random_state=99, stopping_criteria=0,
                   tournament_size=3, transformer='sigmoid', verbose=1,
                   warm_start=False)

In [111]:
y_test = test_df["ADM-DECS"]
X_test = test_df.drop(columns=["ADM-DECS"])

label_test = l_encoder.fit_transform(y_test)
ordinal_test = or_encoder.fit_transform(X_test)

In [112]:
est.score(ordinal_test, label_test)

0.7777777777777778